In [26]:
import tensorflow as tf
from mit_bih_data_maker import mit_bih_data_maker

def test_label_maker(label):
    label_map = {"N":0, "L":1, "R":2, "/":3, "V":4, "A":5}
    if label in label_map:
        return label_map[label]
    return "Omit"

data_maker = mit_bih_data_maker()
data_maker.config("../database/mit-bih/", "../database/mit-bih", "../database/mit-bih-r-label")
#rec_list = ["100", "109", "118", "107", "208", "232"]
rec_list = []
data_maker.load_data(rec_list)

sample_num = -1
label_percent = {0:0.197, 1:0.220, 2:0.191, 3:0.183, 4:0.087, 5:0.122}
train_percent = 0.7
ecg_data = data_maker.make_by_r_peak(before_r = 90, after_r = 146, sample_num = sample_num, label_maker = test_label_maker, \
                                     label_percent = label_percent, train_percent = train_percent)
ecg_data.print_summary()

TRAIN Summay
Sample num: 14473, feature num: 236
Sample num for each label:
Label 0: 2834
Label 1: 3181
Label 2: 2786
Label 3: 2657
Label 4: 1261
Label 5: 1754

TEST Summary
Sample num: 6203, feature num: 236
Sample num for each label:
Label 0: 1239
Label 1: 1368
Label 2: 1163
Label 3: 1127
Label 4: 538
Label 5: 768


In [27]:
import numpy as np

def weight_var(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_var(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 236])
y_ = tf.placeholder(tf.float32, [None])
y_onehot = tf.one_hot(tf.cast(y_, tf.int64), 6, 1.0, 0.0)

#隐层
W_h = weight_var([236, 32])
b_h = bias_var([32])
h = tf.nn.relu(tf.matmul(x, W_h) + b_h)

#输出层
W_o = weight_var([32, 6])
b_o = bias_var([6])
o = tf.nn.softmax(tf.matmul(h, W_o) + b_o)

cross_entropy = -tf.reduce_sum(y_onehot * tf.log(o))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

corr_pred = tf.equal(tf.arg_max(o, 1), tf.arg_max(y_onehot, 1))
accuracy = tf.reduce_mean(tf.cast(corr_pred, tf.float32))

sess.run(tf.initialize_all_variables())

for i in xrange(200000):
    batch = ecg_data.train.next_batch(50)
    
    if i % 1000 == 0:
        train_accuacy = accuracy.eval(feed_dict={x: ecg_data.test.x, y_: ecg_data.test.y})
        print("step %d, training accuracy %g"%(i, train_accuacy))
        
    sess.run(train_step, feed_dict = {x : batch[0], y_ : batch[1]})

step 0, training accuracy 0.057714
step 1000, training accuracy 0.798323
step 2000, training accuracy 0.850717
step 3000, training accuracy 0.865388
step 4000, training accuracy 0.874577
step 5000, training accuracy 0.879736
step 6000, training accuracy 0.887313
step 7000, training accuracy 0.89634
step 8000, training accuracy 0.901822
step 9000, training accuracy 0.90553
step 10000, training accuracy 0.909237
step 11000, training accuracy 0.913751
step 12000, training accuracy 0.915525
step 13000, training accuracy 0.921006
step 14000, training accuracy 0.92552
step 15000, training accuracy 0.927132
step 16000, training accuracy 0.928422
step 17000, training accuracy 0.930517
step 18000, training accuracy 0.929389
step 19000, training accuracy 0.934225
step 20000, training accuracy 0.934064
step 21000, training accuracy 0.935999
step 22000, training accuracy 0.935999
step 23000, training accuracy 0.936321
step 24000, training accuracy 0.938901
step 25000, training accuracy 0.939868
st

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x34a11c90>> ignored


KeyboardInterrupt: 